In [1]:
import piplates.DAQC2plate as DAQC2

In [2]:
# Read Barrier Position
# All the way open
DAQC2.getADC(0,0)-DAQC2.getADC(0,1)

11.802

In [39]:
# Close Barriers
DAQC2.setDAC(0,0,4) # set fast close
DAQC2.setDOUTbit(0,0) # turn on power/start barriers

In [23]:
#stop Barriers
DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers

In [24]:
# Read Barrier Position
# All the way closed
DAQC2.getADC(0,0)-DAQC2.getADC(0,1)

7.789

In [6]:
# Open Barriers
DAQC2.setDAC(0,0,1) # set fast close
DAQC2.setDOUTbit(0,0) # turn on power/start barriers

In [10]:
#stop Barriers
DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers

Change of < 0.2 V for our whole signal. We're looking at a +/- 12 V range at about 13 bit resolution. Should be good to ~ 1% (0.002 V). That may be OK. If reach the advertised 16 bit resolution through averaging then about 10X better.

In [4]:
def barrier_at_limit_check():
    """
    Checks if barrier is at or beyond limit and stops barrier if it is headed moving
    in a direction that would make it worse.
    
    Returns
    =======
    True or False. If True also shuts down power to barrier
    """
    direction = 0 # -1 closing, 0 stopped, 1 openning.
    if (DAQC2.getDAC(0,0)>=2.5):
        direction = -1
    else:
        direction = 1
    position = DAQC2.getADC(0,0)-DAQC2.getADC(0,1)
    if (position >= 7.78) and (direction == -1):
        DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers
        return True
    if (position <= 0.005) and (direction == 1):
        DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers
        return True
    return print("fell through")

In [49]:
DAQC2.getADC(0,0)-DAQC2.getADC(0,1)

8.302

Large fluctuations in barrier position measurement. Problem with voltage supply? Yes, it is being turned off when power to motor is shut off. Needs to be on unswitch +12V line. FIXED.

In [7]:
barrier_at_limit_check()

fell through


In [8]:
DAQC2.getDAC(0,0)

1.0

It is possible to have a separate thread running to check and shut off barrier motion. However, the interface board access does not appear to be thread safe. Collisions occur and then reads or messages to the board throw errors. So all board communication needs to go through a single thread. It could communicate asynchronously with other threads.